# Importing Required Libraries


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

Importing Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset


The dataset we are using is a mix of ...

Naming structure for the files is as follows,

04-P

xx - n

xx- Image number
n - Uploader (G - Ginnie, P - Pocholo)





## Dataset Preparation

Name directories inside google drive

In [6]:
# Google Drive Version
# label_dir = "/content/drive/MyDrive/IAT481/CV_Proj/labels/"
# images_dir = "/content/drive/MyDrive/IAT481/CV_Proj/images/"

# Local Version
label_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\"
images_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\"

In [10]:
# I ran this once to clean up file names (Remove label studio unique ID)
#
# for file in os.listdir(label_dir):
#     if file.endswith(".txt"):
#         test = file.split("-")
#         oldName = label_dir + file;
#         newName = label_dir + test[1] + "-" + test[2]
#         os.rename(oldName,newName)
#         # print(file)

# for file in os.listdir(images_dir):
#     if file.endswith(".jpg"):
#         test = file.split("-")
#         oldName = images_dir + file;
#         newName = images_dir + test[1] + "-" + test[2]
#         os.rename(oldName, newName)

for file in os.listdir(images_dir):
    print(file)

for file in os.listdir(label_dir):
    print(file)

01-G.jpg
01-P.jpg
02-G.jpg
02-P.jpg
03-G.jpg
03-P.jpg
04-G.jpg
04-P.jpg
05-G.jpg
05-P.jpg
06-G.jpg
06-P.jpg
07-G.jpg
07-P.jpg
08-G.jpg
08-P.jpg
09-G.jpg
09-P.jpg
10-G.jpg
10-P.jpg
11-G.jpg
11-p.jpg
12-G.jpg
12-p.jpg
13-G.jpg
13-p.jpg
14-G.jpg
14-P.jpg
15-G.jpg
15-P.jpg
16-G.jpg
16-P.jpg
17-G.jpg
17-P.jpg
18-G.jpg
18-P.jpg
19-G.jpg
19-P.jpg
20-G.jpg
20-P.jpg
21-G.jpg
21-P.jpg
22-G.jpg
22-P.jpg
23-G.jpg
23-P.jpg
24-G.jpg
24-P.jpg
25-G.jpg
25-P.jpg
26-G.jpg
26-P.jpg
27-G.jpg
27-P.jpg
28-G.jpg
28-P.jpg
29-G.jpg
29-P.jpg
30-G.jpg
30-P.jpg
01-G.txt
01-P.txt
02-G.txt
02-P.txt
03-G.txt
03-P.txt
04-G.txt
04-P.txt
05-G.txt
05-P.txt
06-G.txt
06-P.txt
07-G.txt
07-P.txt
08-G.txt
08-P.txt
09-G.txt
09-P.txt
10-G.txt
10-P.txt
11-G.txt
11-p.txt
12-G.txt
12-p.txt
13-G.txt
13-p.txt
14-G.txt
14-P.txt
15-G.txt
15-P.txt
16-G.txt
16-P.txt
17-G.txt
17-P.txt
18-G.txt
18-P.txt
19-G.txt
19-P.txt
20-G.txt
20-P.txt
21-G.txt
21-P.txt
22-G.txt
22-P.txt
23-G.txt
23-P.txt
24-G.txt
24-P.txt
25-G.txt
25-P.txt
26-G.txt
2

Next step is to split the data into training and valuation,

Per instructions we need to use 20/10 split for each dataset, to achieve this I used every picture divisible by 3 (30 images = 10 per person)

In [7]:
images_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\train\\"
images_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\val\\"

label_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\train\\"
label_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\val\\"


In [ ]:
for file in os.listdir(images_dir):
    if file.endswith(".jpg") and int(file.split("-")[0]) % 3 == 0:
        os.rename(images_dir + file, images_dir_val + file)
    else:
        os.rename(images_dir + file, images_dir_train + file)
        
for file in os.listdir(label_dir):
    if file.endswith(".txt") and int(file.split("-")[0]) % 3 == 0:
        os.rename(label_dir + file, label_dir_val + file)
    else:
        os.rename(label_dir + file, label_dir_train + file)

In [9]:
df = {'name': [],
      'label': [],
      'width': [],
      'height': [],
      'x': [],
      'y': [],
     }

test = {'holder': []}

for file in os.listdir(label_dir_train):
    if file.endswith(".txt"):
        content = open(label_dir_train + file)
        readContent = content.readlines()
        for line in readContent:
            clean = line.split(" ")
            df['name'].append(file)
            df['label'].append(clean[0])
            df['x'].append(clean[1])
            df['y'].append(clean[2])
            df['width'].append(clean[3])
            df['height'].append(clean[4])



val = pd.DataFrame(df)
val


,name,label,width,height,x,y
0,01-G.txt,5,0.345558272208639,0.2616136919315404\n,0.5639771801140994,0.2506112469437653
1,01-G.txt,1,0.3162184189079055,0.22982885085574584\n,0.47758761206193967,0.6821515892420539
2,01-P.txt,3,0.11648,0.22200000000000003\n,0.09984,0.505
3,01-P.txt,2,0.13624000000000006,0.26\n,0.23868000000000003,0.496
4,01-P.txt,0,0.10608000000000004,0.1979999999999999\n,0.37439999999999996,0.49700000000000005
...,...,...,...,...,...,...
5588,toonie97_jpeg.rf.993d5be04a3903817914d36aca11e...,5,1,0.9671875,0.5,0.48359375
5589,toonie97_jpeg.rf.a24ff2bf90f52ded6695e3bf63b8c...,5,0.94375,0.9171875,0.471875,0.45859375
5590,toonie9_jpeg.rf.150e384a229f4e3b39dfb760fec311...,5,0.94921875,0.92890625,0.475,0.4765625
5591,toonie9_jpeg.rf.4b2fee7a8a37268dfb557fd6150ed9...,5,1,1,0.5,0.5


# YOLO Training


In [10]:
import ultralytics
from ultralytics import YOLO

In [11]:
model = YOLO('yolov8s.pt')

In [12]:
%%writefile config.yaml

# Google Drive
# path: /content/drive/MyDrive/IAT481/CV_Proj/
# train: /content/drive/MyDrive/IAT481/CV_Proj/images/
# val: /content/drive/MyDrive/IAT481/CV_Proj/images/

# Local
path: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\
train: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\train\\
val: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\val\\

# Classes
nc: 6

# Class Names
names:
  0: Dime
  1: Loonie
  2: Nickel
  3: Penny
  4: Quarter
  5: Toonie

Writing config.yaml


In [14]:
import torch
torch.cuda.is_available()

False

In [ ]:
model.train(data="config.yaml",epochs=3,patience=5,batch=8, lr0=0.0005,imgsz=640)